#### Defined by User

In [ ]:
import os

SEPERATE_SERVING_MONITORING_CLUSTER = False

# IP of the DKube setup where inference will be running 
# without https:// and without port number eg: 1.2.3.4
# It's required to fetch cloudevents secrets
SERVING_DKUBE_IP = "192.168.200.146"

## Dkube information of serving cluster
SERVING_DKUBE_USERNAME = os.getenv("DKUBE_USER_LOGIN_NAME","")

SERVING_DKUBE_TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","")

## Serving Dkube URL with expernal IP
SERVING_DKUBE_URL = "https://192.168.200.146:32222"


## Dkube information of monitoring cluster
## The following values are not required if 
## monitoring and serving is running in same Dkube Cluster
MONITORING_DKUBE_USERNAME = ""

MONITORING_DKUBE_TOKEN = ""

MONITORING_DKUBE_URL = ""

## name of serving cluster
SERVING_DKUBE_CLUSTER_NAME = ""

## Define the model monitor name here that you will be creating 
MONITOR_NAME = "insurance-mm-kf"

# Required in train.ipynb to retrain the model
DKUBE_TRAINING_CODE_NAME = "monitoring-examples"

LIVE_DATASET =  MONITOR_NAME+'-s3'

MINIO_ENDPOINT = f"http://{SERVING_DKUBE_IP}:32221"

TRAINING_DATASET = 'insurance-training-data'
## Inference URL
INFERENCE_URL = None

# dataset to be used as training data
DKUBE_BASE_DATASET = "insurance-data"

# Model name to be created or used for example, it will create the model
# if not existing otherwise it will ignore creation.
MODEL_NAME = MONITOR_NAME

# the frequency with which monitoring will run, value will be considered in minutes
RUN_FREQUENCY = 5

if not(SERVING_DKUBE_TOKEN and SERVING_DKUBE_USERNAME and SERVING_DKUBE_URL and SERVING_DKUBE_IP):
    raise Exception("Please fill the Serving Dkube details first (SERVING_DKUBE_TOKEN, SERVING_DKUBE_URL, SERVING_DKUBE_USERNAME, SERVING_DKUBE_IP)")
    
if (SEPERATE_SERVING_MONITORING_CLUSTER
   and
   not(MONITORING_DKUBE_USERNAME and MONITORING_DKUBE_TOKEN and MONITORING_DKUBE_URL and SERVING_DKUBE_CLUSTER_NAME)):
    raise Exception("Please fill the Monitoring Dkube details first (MONITORING_DKUBE_USERNAME, MONITORING_DKUBE_TOKEN, MONITORING_DKUBE_URL, SERVING_DKUBE_CLUSTER_NAME)")

#### Dkube Resources

In [ ]:
import time,json
from dkube.sdk import *

In [ ]:
serving_api = DkubeApi(URL=SERVING_DKUBE_URL,token=SERVING_DKUBE_TOKEN)
if SERVING_DKUBE_USERNAME == serving_api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

In [ ]:
response =  serving_api.get_cloudevents_logstore_creds()
MINIO_KEY = response["access_key_id"]
MINIO_SECRET_KEY = response["access_key"]
MINIO_BUCKET = response["bucket"]

In [ ]:
data = serving_api.get_modelmonitor_id(MONITOR_NAME)
if data.data:
    MONITOR_ID = data.data.get(MONITOR_NAME)
    if MONITOR_ID:
        raise ValueError(f"{MONITOR_NAME} already existing please use a different name")

In [ ]:
INPUT_TRAIN_TYPE = "training"
cld_config = {"SEPERATE_SERVING_MONITORING_CLUSTER":SEPERATE_SERVING_MONITORING_CLUSTER,
              "MONITOR_NAME":MONITOR_NAME,
              "INPUT_TRAIN_TYPE":INPUT_TRAIN_TYPE, "SERVING_DKUBE_USERNAME":SERVING_DKUBE_USERNAME,
              "SERVING_DKUBE_URL":SERVING_DKUBE_URL, "SERVING_DKUBE_TOKEN":SERVING_DKUBE_TOKEN, 
              "MINIO_KEY":MINIO_KEY,"MINIO_SECRET_KEY":MINIO_SECRET_KEY, "MINIO_ENDPOINT":MINIO_ENDPOINT,
              "MINIO_BUCKET": MINIO_BUCKET,
              "DKUBE_BASE_DATASET":DKUBE_BASE_DATASET, "MODEL_NAME":MODEL_NAME,
              "TRAINING_DATASET":TRAINING_DATASET, "RUN_FREQUENCY":RUN_FREQUENCY,
              "DKUBE_TRAINING_CODE_NAME":DKUBE_TRAINING_CODE_NAME,
              "LIVE_DATASET":LIVE_DATASET,
              "MONITORING_DKUBE_USERNAME":MONITORING_DKUBE_USERNAME, "MONITORING_DKUBE_TOKEN":MONITORING_DKUBE_TOKEN,
              "MONITORING_DKUBE_URL":MONITORING_DKUBE_URL,
              "SERVING_DKUBE_CLUSTER_NAME":SERVING_DKUBE_CLUSTER_NAME}
%store cld_config

#### Dataset

In [ ]:
try:
    dataset = DkubeDataset(SERVING_DKUBE_USERNAME, name=DKUBE_BASE_DATASET)
    dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
    dataset.update_dataset_source(source="pub_url")
    serving_api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")
try:
    dataset = DkubeDataset(SERVING_DKUBE_USERNAME, name=TRAINING_DATASET)
    dataset.update_dataset_source(source="dvs")
    serving_api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")


#### Model

In [ ]:
try:
    model = DkubeModel(SERVING_DKUBE_USERNAME, name=MODEL_NAME)
    model.update_model_source(source='dvs')
    serving_api.create_model(model)
except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e

## Checking for seperate monitoring cluster and adding datasets accordingly.

In [ ]:
if SEPERATE_SERVING_MONITORING_CLUSTER:
    monitoring_api = DkubeApi(URL=MONITORING_DKUBE_URL,token=MONITORING_DKUBE_TOKEN)
    try:
        dataset = DkubeDataset(MONITORING_DKUBE_USERNAME, name=DKUBE_BASE_DATASET)
        dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
        dataset.update_dataset_source(source="pub_url")
        monitoring_api.create_dataset(dataset)
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")

### Model Monitor Datasets

##### Predict and labelled Dataset

In [ ]:
if SEPERATE_SERVING_MONITORING_CLUSTER:
    api = monitoring_api  
    user = MONITORING_DKUBE_USERNAME
else:
    api = serving_api
    user = SERVING_DKUBE_USERNAME

In [ ]:
try:
    dataset = DkubeDataset(user, name=LIVE_DATASET,remote=True)
    dataset.update_dataset_source('s3')
    dataset.update_s3_details(
        endpoint = MINIO_ENDPOINT,
        bucket=MINIO_BUCKET,
        prefix='',
        key=MINIO_KEY,
        secret=MINIO_SECRET_KEY)
    api.create_dataset(dataset)

except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    serving_api.delete_dataset(SERVING_DKUBE_USERNAME,DKUBE_BASE_DATASET,force=True)
    serving_api.delete_dataset(SERVING_DKUBE_USERNAME,TRAINING_DATASET,force=True)
    serving_api.delete_dataset(SERVING_DKUBE_USERNAME,LIVE_DATASET,force=True)
    serving_api.delete_model(DKUBEUSERNAME,MODEL_NAME,force=True)
    %store -d cld_config